# Sistemas de Recomendación - Netflix Prize Challenge

## Filtro Colaborativo a partir de descomposición UV

En este notebook vamos a implementar un sistema de recomendación usando un método tipo descomposición UV.

Vamos a usar la biblioteca Surprise. Te recomendamos tener abierta la [documentación](https://surprise.readthedocs.io/en/stable/getting_started.html) a medida que vas recorriendo esta sección.

### 1. Dataset y Train/test split

Carga de datos

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import pandas as pd

import gc #garbage collector
!pip install surprise

from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/netflix-prize-data/combined_data_1_with_movie_id.csv', dtype={'Rating': np.int8, 'movie_id': np.int16})
print(df1.shape)
df1.head()

(24053764, 3)


,User,Rating,Movie_id
0,1488844,3,1
1,822109,5,1
2,885013,4,1
3,30878,4,1
4,823519,3,1


In [ ]:
df_title = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/netflix-prize-data/movie_titles.csv', encoding = "ISO-8859-1",index_col = 0, header = None, usecols = [0,2], names = ['Movie_Id', 'Name'])
df_title.head()

,Name
Movie_Id,
1,Dinosaur Planet
2,Isle of Man TT 2004 Review
3,Character
4,Paula Abdul's Get Up & Dance
5,The Rise and Fall of ECW


Primero, llevamos el dataset al formato que le gusta a la biblioteca. ¿En qué orden tienen que estar los atributos? Investigar qué hace la clase `Reader` y cuáles son sus parámetros.

In [ ]:
reader = Reader()
#https://surprise.readthedocs.io/en/stable/reader.html
#La clase Reader se utiliza para analizar un archivo que contiene valoraciones.
#Se supone que un archivo de este tipo sólo especifica una calificación por línea, y cada línea debe respetar la siguiente estructura

Luego, creamos el `Dataset` de Surprise usando `Dataset.load_from_df`

In [ ]:
N_filas = 100000 # Limitamos el dataset a N_filas

data = Dataset.load_from_df(df1[['User', 'Movie_id', 'Rating']][:N_filas], reader)

In [ ]:
print(data)

¿Cómo les parece que es mejor hacer el split?¿Dejando películas en test, usuarios o combinaciones?

In [ ]:
trainset, testset = train_test_split(data, test_size=.25)

### 2.2 Entrenamiento

Vamos a entrenar un algoritmo SVD. Explorar sus parámetros y su funcionamiento.

In [ ]:
from surprise import SVD
algo = SVD()
#https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD

Entrenamos sobre el `trainset`

In [ ]:
algo.fit(trainset)

Y predecimos sobre el `testset`. Notar que para predecir sobre un conjunto de test se usa la función `test`.

In [ ]:
predictions = algo.test(testset)

Explorar las característica de `predictions` y alguno de sus elementos

In [ ]:
predictions[1]

Prediction(uid=2471266, iid=25, r_ui=4.0, est=4.181078939233455, details={'was_impossible': False})

En cambio, si queremos predecir para un usuario y una película en particular, usamos la función `predict`

In [ ]:
algo.predict(1328945,28) #usuario, pelicula
#est =prediccion
#el detail la predicción si es posible o no de calcular, si converge o no la cuenta

#r_ui si en el dataset original había un valor para esa pelicula y usuario. r_ui la persona dijo 3 y el est predijo 3.8


Prediction(uid=1328945, iid=28, r_ui=None, est=3.4410687367298056, details={'was_impossible': False})

Exploremos un usuario, veamos cuáles películas le gustaron y cuáles les recomienda el sistema.

Películas que le gustaron

In [ ]:
#crea un dataset para el usuario de los que tienen rating dado
usuario = 1539350
rating = 3   # le pedimos peliculas a las que haya puesto 4 o 5 estrellas
df_user = df1[(df1['User'] == usuario) & (df1['Rating'] >= rating)]
df_user = df_user.reset_index(drop=True)
df_user['Name'] = df_title['Name'].loc[df_user.Movie_id].values
df_user
print(df_user.shape)

(91, 4)


Creamos donde vamos a guardar las recomendaciones

In [ ]:
#no tiene sentido la linea, es solo para achicar los datos
#del data frame es lo que sobra, todo lo que esta en la posición 0 a 4499. lo que sobra.
recomendaciones_usuario = df_title.iloc[:4499].copy()
print(recomendaciones_usuario.shape)
recomendaciones_usuario.head()

(4499, 1)


,Name
Movie_Id,
1,Dinosaur Planet
2,Isle of Man TT 2004 Review
3,Character
4,Paula Abdul's Get Up & Dance
5,The Rise and Fall of ECW


Sacamos del dataframe todas las películas que ya sabemos que vio

In [ ]:
usuario_vistas = df1[df1['User'] == usuario]
print(usuario_vistas.shape)
usuario_vistas.head()

(97, 3)


,User,Rating,Movie_id
219870,1539350,3,33
409717,1539350,4,111
445206,1539350,3,127
664099,1539350,4,175
894718,1539350,4,197


In [ ]:
recomendaciones_usuario.drop(usuario_vistas.Movie_id, inplace = True)
recomendaciones_usuario = recomendaciones_usuario.reset_index()
recomendaciones_usuario.head()

,Movie_Id,Name
0,1,Dinosaur Planet
1,2,Isle of Man TT 2004 Review
2,3,Character
3,4,Paula Abdul's Get Up & Dance
4,5,The Rise and Fall of ECW


Y hacemos las recomendaciones

In [ ]:
recomendaciones_usuario['Estimate_Score'] = recomendaciones_usuario['Movie_Id'].apply(lambda x: algo.predict(usuario, x).est)

In [ ]:
recomendaciones_usuario = recomendaciones_usuario.sort_values('Estimate_Score', ascending=False)
print(recomendaciones_usuario.head(10))

    Movie_Id                                               Name  Estimate_Score
12        13  Lord of the Rings: The Return of the King: Ext...        4.489122
24        25      Inspector Morse 31: Death Is Now My Neighbour        4.030060
4          5                           The Rise and Fall of ECW        3.954948
27        28                                    Lilo and Stitch        3.848400
17        18                                   Immortal Beloved        3.821170
0          1                                    Dinosaur Planet        3.788936
29        30                             Something's Gotta Give        3.725357
2          3                                          Character        3.710083
28        29                                            Boycott        3.685045
22        23  Clifford: Clifford Saves the Day! / Clifford's...        3.624002


### 2. Evaluación

Para el conjunto de `testset`, evaluamos el error RMSE entre las predicciones y las verdaderas calificaciones que le habían dado a las películas. Para eso, buscar en la documentación cómo se hace.

In [ ]:
from surprise import accuracy
#
accuracy.rmse(predictions,verbose=True)


RMSE: 1.0429


1.0429230568805636

### 3. Optimización de parámetros

**Ejercicio**: hacer un gráfico del desempeño del modelo en función del número de factores del `SVD`

In [ ]:
from surprise.model_selection import cross_validate
#https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD
rmse_test_means = []
factores = np.arange(10,300,30)

for factor in factores:
    print(f'\nNúmero de Factores: {factor}')
    algoritmo = SVD(n_factors=factor) #https://surprise.readthedocs.io/en/stable/model_selection.html#cross-validation
    cv = cross_validate(algoritmo, data, measures=['RMSE'], cv = 3, verbose=True) #https://surprise.readthedocs.io/en/stable/model_selection.html
    rmse_test_means.append(np.mean(cv['test_rmse']))

10
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.0430  1.0520  1.0478  1.0476  0.0037  
Fit time          1.72    1.61    1.72    1.68    0.05    
Test time         0.25    0.42    0.23    0.30    0.09    
40
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.0460  1.0477  1.0548  1.0495  0.0038  
Fit time          2.60    2.55    2.60    2.58    0.02    
Test time         0.22    0.22    0.41    0.28    0.09    
70
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.0521  1.0508  1.0495  1.0508  0.0011  
Fit time          3.57    3.57    3.55    3.56    0.01    
Test time         0.22    0.22    0.42    0.29    0.09    
100
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.0489

In [ ]:
plt.scatter(factores,rmse_test_means)
plt.xlabel('Numero de factores')
plt.ylabel('Error RMSE')
plt.show()

**Ejercicio**: recordar que, cuando entrenamos un `SVD`, estamos usando descenso por gradiente para minimizar una función de costo. Usar `GridSearchCV` para buscar valores óptimos para los siguientes parámetros (tres por parámetros, utilizar los valores default de referencia): `n_factors`, `n_epochs`, `lr_all` y `reg_all`. Estudiar qué representa cada uno de ellos mientras esperan. Tomarse un café.

In [ ]:
from surprise.model_selection import GridSearchCV
#n_factors – The number of factors. Default is 100. #numero de componentes principales. en la matriz intermedia (autovalores) tiene tamaño factores r
#n_epochs – The number of iteration of the SGD procedure. Default is 20.
#lr_all – The learning rate for all parameters. Default is 0.005.
#reg_all – The regularization term for all parameters. Default is 0.02.

param_grid = {'n_factors': np.arange(50,200,50),'n_epochs': np.arange(5,20,30), 'lr_all': [0.001,0.005, 0.0001],'reg_all': [0.01,0.02,0.03]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3, n_jobs = -1)
gs.fit(data)


In [ ]:
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

1.0507261877237895
{'n_factors': 50, 'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.03}


## Extra: Agrandando el Dataset

Podemos sumar al dataset el resto de las calificaciones que no usamos.

Como corremos el riesgo de que se nos llene la memoria RAM, vamos a hacerlo de a poco y con cuidado. Arrancamos agregando las calificaciones que hay en `combined_data_2.txt`.

0. Reiniciar el Kernel
1. Abrir el archivo `combined_data_2.txt` con la función `load_data`.
2. Agregar una columna con el `Movie_id` al que corresponden las calificaciones. Si te animas, puedes crear una función que realice este paso.
3. Opcional: filtrar películas con pocas calificaciones
4. Abrir el archivo donde ya está procesado `combined_data_1.txt`. 
5. Agregar al final las nuevas calificaciones y guardarlo en un nuevo archivo.


Una vez que estén contentos con el procedimientos, repetir los pasos anteriores para los archivos faltantes.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import pandas as pd

import gc #garbage collector

In [ ]:
### 1

def load_data(name):
    df = pd.read_csv(name, header = None, names = ['User','Rating'], usecols = [0,1])
    return df


df2 = load_data('/content/drive/MyDrive/Colab Notebooks/netflix-prize-data/combined_data_2.txt')
print(df2.shape)

(26982302, 2)


In [ ]:
df2.head()

,User,Rating
0,4500:,NaN
1,2532865,4.0
2,573364,3.0
3,1696725,3.0
4,1253431,3.0


In [ ]:
### 2.
movies_ids_df2 = df2.User[df2.Rating.isna()].values
print(movies_ids_df2)
print(len(movies_ids_df2))

movies_ids_df2 = np.arange(4500,len(movies_ids_df2) + 4500)
print(movies_ids_df2)

['4500:' '4501:' '4502:' ... '9208:' '9209:' '9210:']
4711
[4500 4501 4502 ... 9208 9209 9210]


In [ ]:
df2_nan = pd.DataFrame(pd.isnull(df2.Rating))
df2_nan = df2_nan[df2_nan['Rating'] == True]
idx_movies_ids = df2_nan.index.values
print(idx_movies_ids)

[       0      259      855 ... 26961403 26980373 26980497]


In [ ]:
# Agregamos el indice de la ultima instancia del dataframe
idx_movies_ids = np.append(idx_movies_ids,df2.shape[0])
cantidad_criticas = np.diff(idx_movies_ids)
cantidad_criticas

array([  259,   596,   105, ..., 18970,   124,  1805])

In [ ]:
columna_movie_id = np.array([])
for i in range(cantidad_criticas.size):
    aux = np.full(cantidad_criticas[i], movies_ids_df2[i])
    columna_movie_id = np.concatenate((columna_movie_id, aux))

In [ ]:
df2['Movie_id'] = columna_movie_id
del columna_movie_id

df2.dropna(inplace = True)
df2['User'] = df2['User'].astype(int)
df2['Movie_id'] = df2['Movie_id'].astype(np.int16)
df2['Rating'] = df2['Rating'].astype(np.int8)

gc.collect()

63

In [ ]:
df2

,User,Rating,Movie_id
1,2532865,4,4500
2,573364,3,4500
3,1696725,3,4500
4,1253431,3,4500
5,1265574,2,4500
...,...,...,...
26982297,2420260,1,9210
26982298,761176,3,9210
26982299,459277,3,9210
26982300,2407365,4,9210


In [ ]:
### 4.

df1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/netflix-prize-data/combined_data_1_with_movie_id.csv', dtype={'Rating': np.int8, 'Movie_id': np.int16})
print(df1.shape)
df1.head()

(24053764, 3)


,User,Rating,Movie_id
0,1488844,3,1
1,822109,5,1
2,885013,4,1
3,30878,4,1
4,823519,3,1


In [ ]:
### 5.

df = df1.copy()
del df1
df = df.append(df2)
print(df.shape)


(51031355, 3)


In [ ]:
print(df)

             User  Rating  Movie_id
0         1488844       3         1
1          822109       5         1
2          885013       4         1
3           30878       4         1
4          823519       3         1
...           ...     ...       ...
26982297  2420260       1      9210
26982298   761176       3      9210
26982299   459277       3      9210
26982300  2407365       4      9210
26982301   627867       3      9210

[51031355 rows x 3 columns]


Chequeamos que estén todas las películas:

In [ ]:
peliculas_presentes = df.Movie_id.unique()
peliculas_presentes

array([   1,    2,    3, ..., 9208, 9209, 9210], dtype=int16)

In [ ]:
print((peliculas_presentes - np.arange(1,9210 + 1)).sum())

0


Y guardamos

In [ ]:
if True:
    df.to_csv('/content/drive/MyDrive/Colab Notebooks/netflix-prize-data/combined_data_1y2_with_movie_id.csv', index= False)